In [ ]:
import h5py
import matplotlib.pyplot as plt
from os import listdir
from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
import numpy as np
import re

axisScale = 0.03
datasets = ['organic', 'gm_late', 'gm_early']

datasetNo = 0

# define storage arrays and reshape to hold all three galaxy type data
redshifts = np.array(range(3*24), dtype=float)
redshifts.shape = (3, 24)
kco_s = np.array(range(3*24), dtype=float)
kco_s.shape = (3, 24)

for dataset in datasets:
    print('-----------------------------------------')
    print(dataset)
    print('-----------------------------------------')
    
    files = listdir('data/' + dataset)

    count = 0
    
    for file in files:
    #if 1==1:



        #file = 'star_particles_015_z002p012.hdf5'
        #file = 'star_particles_028_z000p000.hdf5'
        
        # get redshift from the filename
        m = re.search('(z[0-9])\w+', file)
        s = m.group(0).replace('z', '')
        s = s.replace('p', '.')
        redshift = float(s)
        
        # load data for a particular galaxy at a particular redshift
        f = h5py.File('data/' + dataset + '/' + file,'r')

        # extract data from the file
        ds_c = f['Coordinates']
        ds_v = f['Velocity']
        ds_m = f['Mass']
    

        # Calculate the resultant angular momentum vectors r
        # r contains the angular momentum vectors for each particle per redshift era
        r0 = np.cross(ds_c, ds_v)
        r = np.transpose(np.multiply(ds_m, np.transpose(r0)))
        #print ("shape r", np.shape(r))
        
        # Calculate the total angular momentum vector by summing the vectors (per redshift epoch)
        angMomTot = np.sum(r, axis=0)
        print ("shape angMomTot", np.shape(angMomTot)) 
        
        # Calculate the magnitude of the total angular momentum vector for each redshift epoch
        # We use this to normalise the angular momentum to a unit vector for scaling during the transform
        magnitude = np.linalg.norm(angMomTot)
        
        # Alternative way of calculating the magnitude
        # magnitude2 = np.sqrt(angMomTot[0]**2 + angMomTot[1]**2 + angMomTot[2]**2)
        
        # Calculate the scale factor by working out the angular momentum 
        unitVect_z = angMomTot / magnitude
        #print ("unitVect_z", unitVect_z)
        #print ("np.linalg.norm(unitVect_z)", np.linalg.norm(unitVect_z))
        
        
        # the angular momentum's vector's (unitVect_z) direction is directly out of the plane of the galaxy
        # unitVect_z = k, but j = [-k2/k1, 1, 0], so
        
        k = unitVect_z
        print ("k: ", k)
        
        j = [k[1]/k[0], 1, 0]
        print ("j: ", j)
        
        i = np.cross(j, unitVect_z)
        print ("i: ", i)
        
        
        # transform co-ordinates
        dsc_x_trsfrm = np.dot(ds_c, i)
        #print ("dsc_x_trsfrm", dsc_x_trsfrm)
        #print ("dsc_x_trsfrm shape", np.shape(dsc_x_trsfrm))
        
        dsc_y_trsfrm = np.dot(ds_c, j)
        #print ("dsc_y_trsfrm", dsc_y_trsfrm)
        #print ("dsc_y_trsfrm shape", np.shape(dsc_y_trsfrm))
        
        dsc_z_trsfrm = np.dot(ds_c, k)
        #print ("dsc_z_trsfrm", dsc_z_trsfrm)
        #print ("dsc_z_trsfrm shape", np.shape(dsc_z_trsfrm))
        
        #dsc_trans = list(zip(dsc_x_trsfrm, dsc_y_trsfrm, dsc_z_trsfrm))
        dsc_trans = np.transpose(np.array([dsc_x_trsfrm, dsc_y_trsfrm, dsc_z_trsfrm]))
        
        #transform velocities
        dsv_x_trsfrm = np.dot(ds_v, i)
        #print ("dsv_x_trsfrm", dsv_x_trsfrm)
        #print ("dsv_x_trsfrm shape", np.shape(dsv_x_trsfrm))
        
        dsv_y_trsfrm = np.dot(ds_v, j)
        #print ("dsv_y_trsfrm", dsv_y_trsfrm)
        #print ("dsv_y_trsfrm shape", np.shape(dsv_y_trsfrm))
        
        dsv_z_trsfrm = np.dot(ds_v, k)
        #print ("dsv_z_trsfrm", dsv_z_trsfrm)
        #print ("dsv_z_trsfrm shape", np.shape(dsv_z_trsfrm))
        
        dsv_trans = np.transpose(np.array([dsv_x_trsfrm, dsv_y_trsfrm, dsv_z_trsfrm]))
        
        # Get transposed velocity in the x-y plane only
        ##dsv_trans = np.transpose(np.array([dsv_x_trsfrm, dsv_y_trsfrm, dsv_z_trsfrm]))
        
        #print results
        #print ("dsc_trans", dsc_trans)
        #print ("dsc_trans shape", np.shape(dsc_trans))
        
        #print ("dsc_trans[0,x]")
        #print (dsc_trans[0,0])
        #print ("dsc_trans[0,y]")
        #print (dsc_trans[0,1])
        #print ("dsc_trans[0,z]")
        #print (dsc_trans[0,2])        
        
        #print ("dsv_trans", dsv_trans)
        #print ("dsv_trans shape", np.shape(dsv_trans))
        
        #print ("dsv_trans[0,x]")
        #print (dsv_trans[0,0])
        #print ("dsv_trans[0,y]")
        #print (dsv_trans[0,1])
        #print ("dsv_trans[0,z]")
        #print (dsv_trans[0,2])  
       
        # Calculate KE of transformed particles
        # Get magnitudes of the vectors
        vel_magnitude = np.linalg.norm(dsv_trans, axis=1)

        # Calculate kinetic energy for all star particles
        K_tot = np.sum(0.5 * np.array(ds_m) * np.square(vel_magnitude))
        print ('K_tot: ', K_tot)
        ##specificKe = np.sum(0.5 * np.square(vel_magnitude))


        
        # Calculate R, the distance from the centre in the x-y plane
        R = np.sqrt(np.square(dsc_x_trsfrm) + np.square(dsc_y_trsfrm))
        #print ('R shape', np.shape(R))
        print ('R:', R)
               
        # Calculate momentum 
        res = np.cross(dsc_trans, dsv_trans)
        momentum = np.transpose(np.multiply(ds_m, np.transpose(res)))
        ##momentum = np.multiply(ds_m, res)
        #print('Momentum shape:', np.shape(momentum))
        #print('Momentum',momentum)
        
        # Extract L_z
        L_z = np.array(momentum[:,2])
        #print('L_z:', L_z)    
        
        Krot_co = 0
        for n in range(0, len(ds_m)):
            if L_z[n] < 0 or R[n] == 0:
                continue
            Krot_co = Krot_co + (0.5 * ds_m[n] * np.square(L_z[n] / (ds_m[n]*R[n]) ))
        
        print('Krot_co',Krot_co)
        
        K_co = Krot_co / K_tot
        
        print('K_co',K_co)
        
        redshifts[datasetNo,count] = redshift
        kco_s[datasetNo,count] = K_co
              
        count = count + 1
        print ('-------------------------------------------------')
        
    # Next galaxy
    datasetNo = datasetNo + 1

plt.figure(figsize = (12,10))
plt.title('Kappa Co. vs. Redshift', pad=30)
plt.xlabel('Redshift')
plt.ylabel('Kappa Co.')
plt.ticklabel_format(axis='y', style='sci', useMathText=True)
#plt.semilogy()
for i in range(0,3):
   plt.plot(redshifts[i,0:], kco_s[i,0:])
plt.legend(['organic', 'gm_late', 'gm_early'])
plt.gca().invert_xaxis()
plt.show()      

-----------------------------------------
organic
-----------------------------------------
shape angMomTot (3,)
k:  [0.9515281  0.30598536 0.0311003 ]
j:  [0.32157258995097276, 1, 0]
i:  [ 0.0311003 -0.010001  -0.8531316]
K_tot:  28501792515.963
R: [0.00042708 0.00038965 0.00055176 0.00043276 0.00039526 0.00016764
 0.00040092 0.00039415 0.00032458 0.0001919  0.00072504 0.00108752
 0.00077522 0.00060149 0.000587   0.00035194 0.00018025 0.00045061
 0.00045383 0.00049038 0.00037636 0.00024091 0.00057759 0.00054225
 0.00082936 0.0002818  0.00051389 0.00024078 0.00120068 0.00017892
 0.00057822 0.0005728  0.00031528 0.00043619]
Krot_co 5543794351.718721
K_co 0.1945068664931796
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.65267355 -0.15264495  0.74210293]
j:  [-0.2338764076937788, 1, 0]
i:  [ 0.74210293  0.17356037 -0.6169735 ]
K_tot:  118196994376.26611
R: [3.21167859e-03 1.11177266e-03 2.10258437e-04 1.32340313e-03
 3.38471988e-03 2.09387382e-03 2.21653310

Krot_co 52920287761.71706
K_co 0.19569674146794105
-------------------------------------------------
shape angMomTot (3,)
k:  [-0.24875304  0.17710399 -0.95223742]
j:  [-0.7119671472351409, 1, 0]
i:  [-0.95223742 -0.67796176  0.12266081]
K_tot:  2688153589710.7153
R: [3.32637572e-03 2.85494462e-03 2.93306752e-03 4.08583859e-03
 2.04083869e-03 2.69844325e-03 2.43707353e-03 2.23598181e-03
 2.88277918e-03 2.86019149e-03 2.49495907e-03 2.49890743e-03
 2.14079767e-03 2.01166640e-03 1.79538217e-03 1.90767620e-03
 2.16170114e-03 2.35993660e-03 2.43303058e-03 2.18656318e-03
 2.85073166e-03 3.52470271e-03 3.51319013e-03 3.90012140e-03
 3.99357410e-03 3.51374660e-03 3.69753210e-03 3.76074142e-03
 3.78447669e-03 3.06933837e-03 3.17446877e-03 4.25670544e-03
 4.49931513e-03 4.66862987e-03 3.34101467e-03 4.00099102e-03
 3.65800960e-03 4.30494151e-03 3.64093161e-03 3.43331789e-03
 2.90452823e-03 2.36734309e-03 1.95529735e-03 1.56772802e-03
 1.08801117e-03 1.83954446e-03 2.01722761e-03 1.91038108e-03


Krot_co 674022388377.9606
K_co 0.2507380496999412
-------------------------------------------------
shape angMomTot (3,)
k:  [-0.17999125  0.98364892 -0.00615986]
j:  [-5.464982000756089, 1, 0]
i:  [-0.00615986 -0.03366353 -5.19563242]
K_tot:  30689600018736.727
R: [0.02512477 0.02436837 0.02092983 0.02477473 0.03280416 0.01956308
 0.01744175 0.01121703 0.00831516 0.00846882 0.00941446 0.00910714
 0.01709679 0.01993468 0.01530859 0.01825076 0.01336365 0.01942147
 0.01461687 0.02789207 0.02593264 0.01665397 0.01476969 0.01654915
 0.02907137 0.02907565 0.02600959 0.01808754 0.01712528 0.01334888
 0.0159403  0.01979711 0.01348573 0.02504606 0.03315716 0.02200863
 0.02836674 0.02499829 0.0232221  0.0224831  0.02906184 0.03286398
 0.0239422  0.02774672 0.02478411 0.02571062 0.02360007 0.02936739
 0.03020662 0.03250857 0.02111326 0.02104652 0.02172815 0.01966608
 0.01732306 0.01711319 0.0171659  0.01331507 0.01324467 0.01460224
 0.01415514 0.01346547 0.01528943 0.01578012 0.01510718 0.016283

Krot_co 631434770788395.9
K_co 0.3331485320607282
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.96322618 -0.24176598 -0.11723707]
j:  [-0.25099606488111864, 1, 0]
i:  [-0.11723707 -0.02942604 -0.90254387]
K_tot:  55939029210704.55
R: [2.91905084e-05 4.60378650e-05 4.68978990e-05 ... 7.36155734e-03
 7.51828342e-03 7.73003403e-03]
Krot_co 12912987720711.83
K_co 0.23084039717730367
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.68214507 -0.31826868 -0.65831843]
j:  [-0.46657037823848313, 1, 0]
i:  [-0.65831843 -0.30715188 -0.53365033]
K_tot:  170077505868871.9
R: [0.00656966 0.00836907 0.00696449 ... 0.00034462 0.00036943 0.00036078]
Krot_co 65664869790766.09
K_co 0.3860879159492912
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.57311824 -0.02407833 -0.81911886]
j:  [-0.04201284534483726, 1, 0]
i:  [-0.81911886 -0.03441351 -0.57210665]
K_tot:  642157474899498.2
R: [0.00025157 0.00026473 0.0002

j:  [-3.0393368332265833, 1, 0]
i:  [-0.65716863 -1.99735683  1.94054739]
K_tot:  223227041175.76196
R: [0.00104182 0.0011769  0.00097005 0.00049714 0.00446452 0.01235868
 0.01535053 0.01125985 0.00172388 0.00376937 0.00123679 0.00533437
 0.00714765 0.00710083 0.00940792 0.00878885 0.00160292 0.00450928
 0.00648702 0.00538647 0.00624483 0.00528584 0.00430319 0.00574576
 0.0049058  0.00661529 0.00560212 0.00762444 0.00567205 0.00427325
 0.00446424 0.00416823 0.00553623 0.00304486 0.00377667 0.0019568
 0.00438641]
Krot_co 69573984977.10703
K_co 0.3116736422731449
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.65936841  0.52141242 -0.54162938]
j:  [0.7907755592760853, 1, 0]
i:  [-0.54162938  0.42830727 -0.24704821]
K_tot:  525628185602.7089
R: [7.28149200e-04 3.77277702e-04 1.85505594e-04 2.18731412e-04
 4.65873022e-04 4.82849266e-04 5.42899940e-04 6.99092263e-04
 1.83581962e-03 1.19737664e-03 1.60644203e-03 1.23632435e-03
 3.01822453e-03 2.14792010e-03 1.8

...

Krot_co 436031800708.43494
K_co 0.17990366576806233
-------------------------------------------------
shape angMomTot (3,)
k:  [-0.5965579  -0.79836337  0.08206458]
j:  [1.3382831276977472, 1, 0]
i:  [ 0.08206458 -0.10982564 -0.47187833]
K_tot:  1903608615428.6968
R: [0.0065102  0.00392325 0.00332178 0.00319256 0.00401473 0.00542727
 0.00500464 0.00465328 0.00515881 0.00427294 0.00554252 0.00327823
 0.00101407 0.00249117 0.00427087 0.00327352 0.00296968 0.00278365
 0.00250885 0.00118623 0.00272418 0.00523744 0.0105814  0.00858928
 0.00367237 0.00495956 0.00115071 0.00154115 0.00109387 0.00063425
 0.00952371 0.00987581 0.01020569 0.00930409 0.0075248  0.00341439
 0.00621242 0.00149653 0.00027137 0.00035067 0.00021392 0.0009571
 0.00149124 0.00245447 0.0047     0.00399193 0.0111903  0.01069477
 0.00711091 0.00895812 0.00777189 0.00844961 0.01147175 0.01186778
 0.01411986 0.01442255 0.0115232  0.01242811 0.00920178 0.01022359
 0.00472442 0.00450814 0.00414231 0.00516704 0.00625177 0.00456

Krot_co 40903622800426.17
K_co 0.5462846349399231
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.06822915  0.14876879 -0.9865154 ]
j:  [2.180428603424957, 1, 0]
i:  [-0.9865154   2.15102639  0.25615058]
K_tot:  27180001287639.96
R: [0.01921461 0.01955001 0.01978259 ... 0.01917817 0.02683985 0.02633827]
Krot_co 8938178392442.441
K_co 0.3288512865710222
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.23371699  0.24420484 -0.9411378 ]
j:  [1.0448741736297589, 1, 0]
i:  [-0.9411378   0.98337058  0.02144635]
K_tot:  20793870181190.207
R: [0.01157034 0.01280529 0.01379234 ... 0.02416229 0.01852185 0.01801765]
Krot_co 7946256717775.393
K_co 0.3821441919438088
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.29728551  0.36063997 -0.8840589 ]
j:  [1.2131097840078273, 1, 0]
i:  [-0.8840589   1.0724605   0.14021036]
K_tot:  27830616370579.758
R: [0.00160074 0.00141943 0.00134359 ... 0.01569467 0.01728742 

Krot_co 453936120279.7267
K_co 0.23649707247333604
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.20411564  0.76771383 -0.60741442]
j:  [3.761171100013049, 1, 0]
i:  [-0.60741442  2.28458958  2.68338744]
K_tot:  21168541889640.7
R: [6.49479492e-03 1.20768314e-02 1.77516196e-02 1.83374172e-02
 1.83697066e-02 1.68867998e-02 1.63506726e-02 1.48879244e-02
 2.00102681e-02 3.64820660e-03 1.41639725e-02 1.26635442e-02
 1.32228531e-02 1.02258303e-02 1.04925766e-02 1.15651584e-02
 1.07792205e-02 1.35165145e-02 1.29227845e-02 1.14995811e-02
 1.14110978e-02 1.28575786e-02 1.29289773e-02 9.23959429e-03
 7.88853986e-03 8.97164795e-03 9.67670824e-03 7.88111296e-03
 7.99599605e-03 6.34567891e-03 7.40783174e-03 6.73227695e-03
 7.77792284e-03 7.42396924e-03 8.57001827e-03 8.52791023e-03
 8.36747312e-03 9.72578039e-03 7.68365893e-03 6.73768237e-03
 6.42841127e-03 6.81885326e-03 6.46678673e-03 6.51864057e-03
 5.79681824e-03 8.23747633e-03 1.18667975e-02 1.20844923e-02
 1.0

Krot_co 6214102834957.986
K_co 0.2935536546331043
-------------------------------------------------
shape angMomTot (3,)
k:  [-0.8110438   0.43617788 -0.38981639]
j:  [-0.5377981854668157, 1, 0]
i:  [-0.38981639 -0.20964255  0.57646812]
K_tot:  965837242466.4529
R: [6.34056676e-04 1.01596457e-03 7.83594797e-04 8.67926165e-04
 9.36646080e-04 1.37625658e-03 1.19832835e-03 1.08907301e-03
 2.43886534e-03 1.47188692e-03 1.14378337e-03 1.05577970e-03
 8.44485759e-04 9.00924296e-04 8.14496812e-05 2.01081882e-04
 6.43920125e-04 1.93072848e-03 1.88489404e-03 1.71794272e-03
 1.71660007e-03 2.02737381e-03 1.55547620e-03 2.00067357e-03
 3.36283447e-03 2.30620913e-03 4.77202748e-03 2.28393288e-03
 2.50202172e-03 1.66415749e-03 1.69623925e-03 1.73814831e-03
 1.46987333e-03 1.47163662e-03 1.94725231e-03 1.52754717e-03
 1.93778426e-03 2.63929502e-03 2.16837299e-03 2.13643734e-03
 2.20357063e-03 3.60268369e-03 3.66633715e-03 2.85422960e-03
 2.81773025e-03 2.50568030e-03 2.21734674e-03 2.74364872e-03
 3

Krot_co 12773224447923.543
K_co 0.3652125554943226
-------------------------------------------------
shape angMomTot (3,)
k:  [0.5962591  0.76582492 0.24080548]
j:  [1.2843827861537893, 1, 0]
i:  [ 0.24080548 -0.30928641  0.38735325]
K_tot:  17396175000969.691
R: [0.00165077 0.0018635  0.00283955 ... 0.00217269 0.00226854 0.00216416]
Krot_co 2370227224564.473
K_co 0.1362499069153048
-------------------------------------------------
shape angMomTot (3,)
k:  [ 0.59239507 -0.65443832  0.4698708 ]
j:  [-1.104732896713157, 1, 0]
i:  [0.4698708  0.51908173 0.13058447]
K_tot:  272735653106337.38
R: [0.00022247 0.00011252 0.00018039 ... 0.00300094 0.00273394 0.00320274]
